In [0]:
# pip install pyarrow==3.0.0
!pip install pyarrow --upgrade
import pyarrow
pyarrow.__version__

Requirement already up-to-date: pyarrow in /databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages (3.0.0)
Requirement already satisfied, skipping upgrade: numpy>=1.16.6 in /databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages (from pyarrow) (1.18.1)
Out[5]: '3.0.0'

In [0]:
from sklearn.datasets import make_classification as mc
import pandas as pd


sample = .01
#training
data,y = mc(n_samples=500_000_000,n_features= 5,n_informative=3,weights=(.5,.50)) #10_000_000
data = pd.DataFrame(data)
y = pd.Series(y,name='stabf')
data = pd.concat((data,y),axis=1)
df_train_spark = spark.createDataFrame(data)

#validation
# data_val,y_val = mc(n_samples=10000,n_features= 5,n_informative=3,weights=(.7,.30)) #10_000_000
# data_val = pd.DataFrame(data_val)
# y_val = pd.Series(y,name='stabf')
# data_val = pd.concat((data_val,y_val),axis=1)
# df_val_spark = spark.createDataFrame(data_val)
df_val_spark = df_train_spark.stat.sampleBy("stabf",fractions={0:sample,1:sample})


# test
# data_t,y_t = mc(n_samples=100,n_features= 5,n_informative=3) #10_000_000
# data_t = pd.DataFrame(data_t)
# y_t = pd.Series(y_t,name='stabf')
# data_t = pd.concat((data_t,y_t),axis=1)
# df_test_spark = spark.createDataFrame(data_t)
df_test_spark = df_train_spark.stat.sampleBy("stabf",fractions={0:sample,1:sample})

# drop val ad test from train
df_train_spark = df_train_spark.subtract(df_test_spark.union(df_val_spark))
# drop y column from test
df_test_spark = df_test_spark.drop('stabf')

del(data)



In [0]:
# df_train_spark.rdd.getNumPartitions()
# Repartition data

df_train_spark = df_train_spark.repartition(8)
df_val_spark = df_val_spark.repartition(8)
df_test_spark = df_test_spark.repartition(8)

Out[26]: 100

In [0]:
df_train_spark.count()
spark.sql.exe
spark.conf.set('spark.sql.execution.arrow.maxRecordsPerBatch', 20000)
park.sql.execution.arrow.maxRecordsPerBatch

Out[20]: 98010297

In [0]:
# add partetion ID column 
from pyspark.sql.functions import spark_partition_id, asc, desc
train =df_train_spark.withColumn("partitionId", spark_partition_id())
train.show()

+-------------------+--------------------+--------------------+--------------------+--------------------+-----+-----------+
 0| 1| 2| 3| 4|stabf|partitionId|
+-------------------+--------------------+--------------------+--------------------+--------------------+-----+-----------+
 1.6132983899268334| 0.32755282284814247| 1.2663883318340718| -1.3087938970752426|-0.16067396927938038| 0| 0|
 1.4162434292969723| 1.2022677302337044| -0.6312400911955276| -1.8502588814330654| 1.7254420882156167| 0| 0|
 1.6047203403822705| 0.36899575381874467| 0.7197984768321403| -2.0556168912068| -0.3497334712646718| 0| 0|
 1.7774966218306572| 0.2556831376800686| 1.667776674468303| -1.2478866521730057|-0.34793375343806243| 0| 0|
 1.321466774816951| 0.3536677768890217|-0.16414290540460774| -2.7735885772278843| -0.5886163219193867| 0| 0|
 -0.961909320032657| -0.8215038199876659| 0.9145719759739058| 2.01087247192286| -0.8924662966523819| 0| 0|
-0.5918018932480501|-0.05543091054566792| -0.9731884298032003|-0.17639175707201638|-0.04311731848285705| 1| 0|
 1.1400536221864623| 0.780211398265424| -0.4403673879795609| -1.8018421271421166| 0.8301496676120891| 0| 0|
 1.3063103129566087| 0.7925968075312332| 0.09930769793705385| -1.3400279031289148| 0.9938649798200674| 0| 0|
 1.9118586213384283| 0.865201913291059| 1.3635600670785875| -0.7012216852647235| 1.2519161998584765| 0| 0|
 -4.42378357425031| -2.038826681780526| -2.6869286666330576| 2.2775204575130306| -2.7302038423902166| 1| 0|
 -1.282645452971698| 0.3334486509597984| -0.8073491782781673| 2.6816459897175795| 2.1483174017600666| 1| 0|
-0.7256527743022798| -0.5660479680092607| 0.8595593118393215| 1.9040739872534922|-0.39850324538444126| 1| 0|
 1.063020941799655| 0.666467356583469|-0.35427219012354594| -1.727710052068173| 0.6131128786826082| 0| 0|
-0.7643246420881217| 0.3152747655674238| -1.4708311996494772| 0.2996240710430299| 1.0516245456806157| 1| 0|
 2.391414423727838| 0.5992514179350789| 3.4675190570512595| 0.8188041524531913| 1.0918527720189855| 0| 0|
 1.0591780502372934| 0.6102359860484173| 0.2147623870488049| -0.94751166122542| 0.7837397365986375| 0| 0|
 2.168739308020944| 0.08974454783227746| 2.504609144445231| -1.2792273556074694| -0.8498039893326197| 0| 0|
 2.36437588237519| 0.802145062850763| 2.6633930818255545| 0.07327295823490343| 1.2853046894940192| 0| 0|
 0.6602221132114848|-0.07489735402082491| 0.911530322827065|-0.38305141622650896| -0.4950526327987086| 0| 0|
+-------------------+--------------------+--------------------+--------------------+--------------------+-----+-----------+
only showing top 20 rows

In [0]:
# check label ratio in each partetion
from typing import Iterator
import pandas as pd

def pandas_filter(iterator: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
  for pdf in iterator:
    id = pdf['partitionId']
    pdf = pd.DataFrame(pdf.groupby('stabf').size()).reset_index().rename({0:"count"},axis=1)
    pdf['part_id'] = id
#     pdf['part_id'] = spark_partition_id()
    yield pdf
    
train.mapInPandas(pandas_filter,spark.createDataFrame(pd.DataFrame(dict(stabf=[0],count=[0],part_id=[1]))).schema).show(10000)
# df_train_spark.mapInPandas(pandas_filter,spark.createDataFrame(pd.DataFrame(dict(stabf=[0],count=[0]))).schema).show(1000)


+-----+-----+-------+
stabf|count|part_id|
+-----+-----+-------+
 0| 4952| 0|
 1| 5048| 0|
 0| 4990| 0|
 1| 5010| 0|
 0| 5051| 0|
 1| 4949| 0|
 0| 5010| 0|
 1| 4990| 0|
 0| 5000| 0|
 1| 5000| 0|
 0| 4952| 0|
 1| 5048| 0|
 0| 5082| 0|
 1| 4918| 0|
 0| 4935| 0|
 1| 5065| 0|
 0| 4896| 0|
 1| 5104| 0|
 0| 5062| 0|
 1| 4938| 0|
 0| 5025| 0|
 1| 4975| 0|
 0| 4985| 0|
 1| 5015| 0|
 0| 5045| 0|
 1| 4955| 0|
 0| 5053| 0|
 1| 4947| 0|
 0| 4950| 0|
 1| 5050| 0|
 0| 5079| 0|
 1| 4921| 0|
 0| 5051| 0|
 1| 4949| 0|
 0| 4985| 0|
 1| 5015| 0|
 0| 4931| 0|
 1| 5069| 0|
 0| 5019| 0|
 1| 4981| 0|
 0| 4928| 0|
 1| 5072| 0|
 0| 5102| 0|
 1| 4898| 0|
 0| 4932| 0|
 1| 5068| 0|
 0| 4986| 0|
 1| 5014| 0|
 0| 5049| 0|
 1| 4951| 0|
 0| 4998| 0|
 1| 5002| 0|
 0| 4979| 0|
 1| 5021| 0|
 0| 5021| 0|
 1| 4979| 0|
 0| 5041| 0|
 1| 4959| 0|
 0| 5054| 0|
 1| 4946| 0|
 0| 5062| 0|
 1| 4938| 0|
 0| 4960| 0|
 1| 5040| 0|
 0| 5047| 0|
 1| 4953| 0|
 0| 5046| 0|
 1| 4954| 0|
 0| 5053| 0|
 1| 4947| 0|
 0| 5003| 0|
 1| 4997| 0|
 0| 5044| 0|
 1| 4956| 0|
 0| 4920| 0|
 1| 5080| 0|
 0| 4984| 0|
 1| 5016| 0|
 0| 4968| 0|
 1| 5032| 0|
 0| 4983| 0|
 1| 5017| 0|
 0| 5020| 0|
 1| 4980| 0|
 0| 5036| 0|
 1| 4964| 0|
 0| 5080| 0|
 1| 4920| 0|
 0| 5016| 0|
 1| 4984| 0|
 0| 5010| 0|
 1| 4990| 0|
 0| 5070| 0|
 1| 4930| 0|
 0| 4962| 0|
 1| 5038| 0|
 0| 4995| 0|
 1| 5005| 0|
 0| 4934| 0|
 1| 5066| 0|
 0| 4998| 0|
 1| 5002| 0|
 0| 4890| 0|
 1| 5110| 0|
 0| 4903| 0|
 1| 5097| 0|
 0| 5051| 0|
 1| 4949| 0|
 0| 5119| 0|
 1| 4881| 0|
 0| 5041| 0|
 1| 4959| 0|
 0| 5034| 0|
 1| 4966| 0|
 0| 4952| 0|
 1| 5048| 0|
 0| 4987| 0|
 1| 5013| 0|
 0| 5006| 0|
 1| 4994| 0|
 0| 4971| 0|
 1| 5029| 0|
 0| 4993| 0|
 1| 5007| 0|
 0| 4959| 0|
 1| 5041| 0|
 0| 5107| 0|
 1| 4893| 0|
 0| 4940| 0|
 1| 5060| 0|
 0| 5008| 0|
 1| 4992| 0|
 0| 4980| 0|
 1| 5020| 0|
 0| 4921| 0|
 1| 5079| 0|
 0| 4873| 0|
 1| 5127| 0|
 0| 4917| 0|
 1| 5083| 0|
 0| 4971| 0|
 1| 5029| 0|
 0| 4975| 0|
 1| 5025| 0|
 0| 5126| 0|
 1| 4874| 0|
 0| 4984| 0|
 1| 5016| 0|
 0| 4960| 0|
 1| 5040| 0|
 0| 5045| 0|
 1| 4955| 0|
 0| 5024| 0|
 1| 4976| 0|
 0| 5037| 0|
 1| 4963| 0|
 0| 5032| 0|
 1| 4968| 0|
 0| 5102| 0|
 1| 4898| 0|
 0| 4980| 0|
 1| 5020| 0|
 0| 4984| 0|
 1| 5016| 0|
 0| 4970| 0|
 1| 5030| 0|
 0| 4985| 0|
 1| 5015| 0|
 0| 4954| 0|
 1| 5046| 0|
 0| 4978| 0|
 1| 5022| 0|
 0| 5095| 0|
 1| 4905| 0|
 0| 5097| 0|
 1| 4903| 0|
 0| 5025| 0|
 1| 4975| 0|
 0| 4975| 0|
 1| 5025| 0|
 0| 5057| 0|
 1| 4943| 0|
 0| 5002| 0|
 1| 4998| 0|
 0| 5071| 0|
 1| 4929| 0|
 0| 5016| 0|
 1| 4984| 0|
 0| 4978| 0|
 1| 5022| 0|
 0| 5026| 0|
 1| 4974| 0|
 0| 5066| 0|
 1| 4934| 0|
 0| 4984| 0|
 1| 5016| 0|
 0| 4952| 0|
 1| 5048| 0|
 0| 4968| 0|
 1| 5032| 0|
 0| 4976| 0|
 1| 5024| 0|
 0| 5009| 0|
 1| 4991| 0|
 0| 4992| 0|
 1| 5008| 0|
 0| 4950| 0|
 1| 5050| 0|
 0| 5022| 0|
 1| 4978| 0|
 0| 5011| 0|
 1| 4989| 0|
 0| 4960| 0|
 1| 5040| 0|
 0| 4977| 0|
 1| 5023| 0|
 0| 5033| 0|
 1| 4967| 0|
 0| 4991| 0|
 1| 5009| 0|
 0| 5029| 0|
 1| 4971| 0|
 0| 4953| 0|
 1| 5047| 0|
 0| 5001| 0|
 1| 4999| 0|
 0| 4997| 0|
 1| 5003| 0|
 0| 4961| 0|
 1| 5039| 0|
 0| 4936| 0|
 1| 5064| 0|
 0| 5102| 0|
 1| 4898| 0|
 0| 4915| 0|
 1| 5085| 0|
 0| 4991| 0|
 1| 5009| 0|
 0| 4931| 0|
 1| 5069| 0|
 0| 5006| 0|
 1| 4994| 0|
 0| 5014| 0|
 1| 4986| 0|
 0| 4984| 0|
 1| 5016| 0|
 0| 5018| 0|
 1| 4982| 0|
 0| 4968| 0|
 1| 5032| 0|
 0| 5015| 0|
 1| 4985| 0|
 0| 5053| 0|
 1| 4947| 0|
 0| 4974| 0|
 1| 5026| 0|
 0| 4959| 0|
 1| 5041| 0|
 0| 5074| 0|
 1| 4926| 0|
 0| 4915| 0|
 1| 5085| 0|
 0| 5034| 0|
 1| 4966| 0|
 0| 5006| 0|
 1| 4994| 0|
 0| 5012| 0|
 1| 4988| 0|
 0| 4975| 0|
 1| 5025| 0|
 0| 5058| 0|
 1| 4942| 0|
 0| 4961| 0|
 1| 5039| 0|
 0| 5021| 0|
 1| 4979| 0|
 0| 5022| 0|
 1| 4978| 0|
 0| 4977| 0|
 1| 5023| 0|
 0| 4978| 0|
 1| 5022| 0|
 0| 5032| 0|
 1| 4968| 0|
 0| 4993| 0|
 1| 5007| 0|
 0| 4996| 0|
 1| 5004| 0|
 0| 5045| 0|
 1| 4955| 0|
 0| 4996| 0|
 1| 5004| 0|
 0| 4965| 0|
 1| 5035| 0|
 0| 4989| 0|
 1| 5011| 0|
 0| 4969| 0|
 1| 5031| 0|
 0| 5044| 0|
 1| 4956| 0|
 0| 4993| 0|
 1| 5007| 0|
 0| 5065|

In [0]:
from pyspark.sql.functions import spark_partition_id, asc, desc
train\
    .groupBy("partitionId")\
    .count()\
    .orderBy(asc("count"))\
    .show(200)

+-----------+--------+
partitionId| count|
+-----------+--------+
 5|12251243|
 4|12251255|
 3|12251268|
 2|12251276|
 1|12251290|
 0|12251311|
 7|12251322|
 6|12251332|
+-----------+--------+

In [0]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd
from typing import Iterator
from pycaret.classification import *
from sklearn import metrics as m
import pickle
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
import time
from pyspark.ml.evaluation import MulticlassClassificationEvaluator as mult
from sklearn.metrics import roc_auc_score

class Binary_Classifier():
  
  def __init__(self,data:DataFrame,
    target_variable:str,
    validation_percentage:float=0.1,
    validation_data:DataFrame =None, 
    batch_size:int =3,
    prediction_threshold:float=0.50,
    session_id:int=None,
    gpu_training:bool=False):
    
    """
    Docs String Here
    """
    
    self.training_data = data
    self.target_variable = target_variable
    self.batch_size= batch_size
    self.validation_data = validation_data
    self.validation_percentage = validation_percentage
    self.prediction_threshold = prediction_threshold
    self.gpu_training = gpu_training
    self.session_id = session_id
    self.validation_data = validation_data
    self._data_after_preprocessing = None
    self.schema = self.training_data.schema
    
  
  def select_model(self,model_name:str,**custom_parameters):
    """
    Doc String
    """
    self.model_name = model_name
    self.custom_parameters = custom_parameters
    
    
  def auto_select_model(self):
    """
    Doc String
    """
    self.model_name = "Auto Select"

    
  
  def fit_predict(self,blend=False):
    
    # make/check tarin & validate data
    #=================================
    # future proprocessign step
    
    start = time.perf_counter()
    if self.validation_data:
      self.validation_data = self.validation_data
    else:
      self.validation_data = self.training_data.stat.sampleBy("stabf",{0:self.validation_percentage,1:self.validation_percentage},seed=i) # hard coded - labels & seed
      # remove from training
      self.training_data = self.training_data.subtract(self.validation_data)
      
    
    #make samples
    #============
    fraction = 1/self.batch_size
    # ph
    df_sample = spark.createDataFrame(data=[],schema=self.training_data.schema)
    df_sample = df_sample.withColumn("sample_index",lit(0))
    self.training_data= self.training_data.withColumn("sample_index",lit(0))
    self.group_column = "sample_index"

    for i in range(self.batch_size):
      sample = self.training_data.stat.sampleBy("stabf",{0:fraction,1:fraction},seed=i) # hard coded - labels & seed
      sample = sample.withColumn("sample_index",lit(i))
      df_sample = df_sample.union(sample)
    
    # save the sampled data
    self.df_sample = df_sample
    self.df_sample.count()
    end = time.perf_counter()
    print("processing took {0} minutes to run ...".format((end-start)/60))

    
    #Training output Scheema
    #=======================
    start = time.perf_counter()
    self.internal_training_schema = StructType([StructField("model_id", LongType(), True),  
                       StructField("model_returend", BinaryType(), True),
                       StructField("Accuracy",FloatType(), True),
                       StructField("AUC",FloatType(), True)])
  
    
    # make training function
    #======================
    # this function will not take class attributes , so we will need to give it local/enclosing variables
    group_column = self.group_column
    target = self.target_variable
    sessionid = self.session_id
    gpu = self.gpu_training
    model_name = self.model_name
    
    # if custome model only then make cp
    if model_name != "Auto Select":
      cp_create_model = self.custom_parameters
    
    def internal_training(key,pdf):
      # get the sample id
      id = pdf[group_column].iloc[0]
      # create a smaller dataset
      small = pdf.head(100)
      #call setup
      cls1= setup(small.drop([group_column],axis=1), target = target, session_id=sessionid,silent=True,html=False,preprocess=False,use_gpu=gpu)
      # Now change the it with original (since we do not need preprocessing)
      set_config('X_train',pdf.drop([group_column,target],axis=1))
      set_config('y_train',pdf[target])
      
      #Modeling
      # if create model is selected
      if model_name == "Auto Select":
        model = compare_models(n_select=1,cross_validation=False) 
      else:
        model = create_model(model_name ,cross_validation=False,**cp_create_model) # while unpacking the custom parameters
        
      
      
      # try to get AUC
      try:
        prediction_prob = model.predict_proba (pdf.drop([group_column,target],axis=1))
        AUC = m.roc_auc_score(pdf[target],prediction_prob[:, 1])
      except:
        AUC = 0.0
      
      
      # get prediction & accuracy
      prediction = model.predict(pdf.drop([group_column,target],axis=1))  
      accuracy = m.accuracy_score(y_true=pdf[target],y_pred=prediction)
      # convert to df
      return_df = pd.DataFrame(dict(model_id = id,model_returend = [pickle.dumps(model)], Accuracy = accuracy, AUC = AUC))
     

      return(return_df)

    
    # Call the training function with sampled data
    #=============================================
    return_internal_training  = self.df_sample.groupby(self.group_column).applyInPandas(internal_training, schema=self.internal_training_schema)
          
    return_internal_training.count()
    end = time.perf_counter()
    print("internal training took {0} minutes to run ...".format((end-start)/60))
    
    self.internal_training_res = return_internal_training
    
    # Collect trained models in a dic
    #================================
    start = time.perf_counter()
    dict_models ={}
    for r in return_internal_training.rdd.collect():
      dict_models[r['model_id']] = pickle.loads(r['model_returend'])
    
    end = time.perf_counter()
    print("collecting models took {0} minutes to run ...".format((end-start)/60))
    
    # Apply the second Level training 
    #================================
    #================================
    
    
    # First, creat a schema for training
    # outcome (predictions)
    #==================================
    
    # this has to be flexable, depending upon the 
    # batches we made 
    # easiest way is to creat a dummy pandas df 
    # with required column name and data types (always float)
    dummy = pd.DataFrame(dict(zip (list(range(self.batch_size)) , [[i*1.0,i*1.0] for i in range(self.batch_size)])))
    # add one extra column named labels
    dummy['label'] = 0.0
    # get the schema
    schema2 = spark.createDataFrame(dummy).schema
    
    
    # creat another pandas udf 
    # for generating predictions
    # on entire trainingTRAINING  data set
    #===========================
    # this will not take any instances
    def training_prediction_all(iterator: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
      for pdf in iterator:
        res = pd.DataFrame()
        for k in dict_models.keys():
          try:
            pre = dict_models[k].predict_proba(pdf.drop([group_column,target],axis=1))
            res[str(k)] = pre[:, 1]
          except:
            pre = dict_models[k].predict(pdf.drop([group_column,target],axis=1))
            res[str(k)] = pre

        # put the prediction with the results
        res['label'] = pdf[target]
        yield (res)
    
    start = time.perf_counter()
    return_internal_training_second = self.df_sample.mapInPandas(training_prediction_all, schema=schema2)
    return_internal_training_second.count()
    end = time.perf_counter()
    
    print("predicion on entire training data took {0} minutes to run ...".format((end-start)/60))
    
    #Validation Predictionson
    # ========================
    #return predictions on entire validation data 
    # Do we need this ? remmeber we have OOB predictions from training data
    def validation_prediction_all(iterator: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
      for pdf in iterator:
        res = pd.DataFrame()
        for k in dict_models.keys():
          try:
            pre = dict_models[k].predict_proba(pdf.drop([group_column,target],axis=1,errors='ignore'))
            res[str(k)] = pre[:, 1]
          except:
            pre = dict_models[k].predict(pdf.drop([group_column,target],axis=1,errors="ignore"))
            res[str(k)] = pre

        # put the prediction with the results
        res['label'] = pdf[target]
        yield (res)
    
    start = time.perf_counter()
    return_internal_validation_second = self.validation_data.mapInPandas(validation_prediction_all, schema=schema2)
    return_internal_validation_second.count()
    end = time.perf_counter()
    print("predicion on entire validation data took {0} minutes to run ...".format((end-start)/60))
    
    #Now train on second dataset with spark
    # # only IF Blend is set to True
    # So far only selecting lr
    #=======================================
    if blend:
      mlib_lr = LogisticRegression()
      # make feature vectors
      vec = VectorAssembler(inputCols=[str(i) for i in range(self.batch_size)],outputCol='features')
      # now transform
      df_training = vec.transform(return_internal_training_second)
      df_validation = vec.transform(return_internal_validation_second)

      # only select feature vector and target column

      df_training = df_training.select("features","label")


      df_validation = df_validation.select("features","label")

      # now train
      start = time.perf_counter()
      fitted_model = mlib_lr.fit(df_training)
      end = time.perf_counter()
      print("spark model fitting took {0} minutes to run ...".format((end-start)/60))

      start = time.perf_counter()    
      Evaluated = fitted_model.evaluate(df_validation)
      end = time.perf_counter()
      print("spark model evaluation took {0} minutes to run ...".format((end-start)/60))

      # make a data frame
      start = time.perf_counter()
      results = pd.DataFrame(dict(Accuracy=[fitted_model.summary.accuracy,Evaluated.accuracy],AUC=[fitted_model.summary.areaUnderROC,Evaluated.areaUnderROC]),index=['Training','Validation'])
      end = time.perf_counter()
      print("spark results took {0} minutes to run ...".format((end-start)/60))

      self.train_res = fitted_model
      self.val_res = Evaluated
      
      
      
    # # only IF Blend is set to False
    # Then simple average with soft voting
    # So far only selecting lr
    #=======================================
    if not blend:
      
      # FOR TRAINING DATA SET
      # we need the table plus two more columns (score & voted prediction)
      return_internal_training_second = return_internal_training_second.withColumn("Average_Score",lit(0.0))
      return_internal_validation_second = return_internal_validation_second.withColumn("Average_Score",lit(0.0))
      
      l = self.batch_size
      cols = return_internal_training_second.columns[:-3]
      def pandas_filter(iterator: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
          for pdf in iterator:
            pdf['Average_Score'] = pdf[cols].mean(axis=1)
            yield pdf
      
      # For Training
      #-------------
      start = time.perf_counter()
      training_predictions  = return_internal_training_second.mapInPandas(pandas_filter, schema=return_internal_training_second.schema)
      # get prediction logic
      training_predictions = training_predictions.withColumn("Prediction",when(training_predictions.Average_Score>=self.prediction_threshold,1.0).otherwise(0.0))
      self.training_predictions = training_predictions
      self.training_predictions.count()
      end = time.perf_counter()
      print("voting on training data took {0} minutes to run ...".format((end-start)/60))
      
      
      # For Validation
      #---------------
      start = time.perf_counter()
      validation_predictions  = return_internal_validation_second.mapInPandas(pandas_filter, schema=return_internal_validation_second.schema)
      # get prediction logic
      validation_predictions = validation_predictions.withColumn("Prediction",when(validation_predictions.Average_Score>=self.prediction_threshold,1.0).otherwise(0.0))
      self.validation_predictions = validation_predictions
      self.validation_predictions.count()
      end = time.perf_counter()
      print("voting on validation data took {0} minutes to run ...".format((end-start)/60))
      
      # Accuracy, recall, precision , F1 will be calculated through multicalss evaluator 
      # AUC will be calculated through sklearn 
      training_accuracy = mult(predictionCol='Prediction',probabilityCol='Average_Score',metricName='accuracy').evaluate(self.training_predictions)
      validation_accuracy =  mult(predictionCol='Prediction',probabilityCol='Average_Score',metricName='accuracy').evaluate(self.validation_predictions)
      # define AUC function
      @pandas_udf("double")
      def pandas_auc(label: pd.Series,probability: pd.Series) -> float:
        score = roc_auc_score(y_true= label, y_score=probability)
        return score
      
      training_auc = self.training_predictions.select(pandas_auc('label','Average_Score')).first()[0]
      validation_auc = self.validation_predictions.select(pandas_auc('label','Average_Score')).first()[0]
      
      #m_precision = m(predictionCol='Prediction',probabilityCol='Average_score',metricName='precisionByLabel').evaluate((cls.training_predictions))
      #recall=  m(predictionCol='Prediction',probabilityCol='Average_score',metricName='weightedRecall').evaluate((cls.training_predictions))
      start = time.perf_counter()
      results = pd.DataFrame(dict(Accuracy=[training_accuracy,validation_accuracy],AUC=[training_auc,validation_auc]),index=['Training','Validation'])
      end = time.perf_counter()
      print("results took {0} minutes to run ...".format((end-start)/60))
      
    
        
    return(results)
    
    
    
    
  

In [0]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd
from typing import Iterator
from pycaret.classification import *
from sklearn import metrics as m
import pickle
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
import time
from pyspark.ml.evaluation import MulticlassClassificationEvaluator as mult
from sklearn.metrics import roc_auc_score

class Binary_Classifier():
  
  def __init__(self,data:DataFrame,
    target_variable:str,
    validation_percentage:float=0.1,
    validation_data:DataFrame =None, 
    batch_size:int =3,
    prediction_threshold:float=0.50,
    session_id:int=None,
    gpu_training:bool=False,
    stratified=False,
    _count=True):
    
    """
    Docs String Here
    """
    
    self.training_data = data
    self.target_variable = target_variable
    self.batch_size= batch_size
    self.validation_data = validation_data
    self.validation_percentage = validation_percentage
    self.prediction_threshold = prediction_threshold
    self.gpu_training = gpu_training
    self.session_id = session_id
    self.validation_data = validation_data
    self._data_after_preprocessing = None
    self.schema = self.training_data.schema
    self.stratified = stratified
    self._count = _count
    
  
  def select_model(self,model_name:str,**custom_parameters):
    """
    Doc String
    """
    self.model_name = model_name
    self.custom_parameters = custom_parameters
    
    
  def auto_select_model(self):
    """
    Doc String
    """
    self.model_name = "Auto Select"

    
  
  def fit_predict(self,blend=False):
    
    self.blend_option = blend
    
    # make/check tarin & validate data
    #=================================
    # future proprocessign step
    
    
    
    if self.validation_data:
      self.validation_data = self.validation_data
    else:
      self.validation_data = self.training_data.stat.sampleBy("stabf",{0:self.validation_percentage,1:self.validation_percentage},seed=i) # hard coded - labels & seed
      # remove from training
      self.training_data = self.training_data.subtract(self.validation_data)
      
    
    # if user wants to stratify:
    #make samples
    #============
    if self.stratified:
      start = time.perf_counter()
      fraction = 1/self.batch_size
      # ph
      df_sample = spark.createDataFrame(data=[],schema=self.training_data.schema)
      df_sample = df_sample.withColumn("sample_index",lit(0))
      self.training_data= self.training_data.withColumn("sample_index",lit(0))
      self.group_column = "sample_index"

      for i in range(self.batch_size):
        sample = self.training_data.stat.sampleBy("stabf",{0:fraction,1:fraction},seed=i) # hard coded - labels & seed
        sample = sample.withColumn("sample_index",lit(i))
        df_sample = df_sample.union(sample)

      # save the sampled data
      self.df_sample = df_sample
      if self._count:
        self.df_sample.count()
      
      end = time.perf_counter()
      print("sampling took {0} minutes to run ...".format((end-start)/60))
    else:
      # Else , rename training data to sample data
      self.df_sample =  self.training_data 
    
    
    #Training output Scheema
    #=======================
    
    self.internal_training_schema = StructType([StructField("model_id", LongType(), True),  
                       StructField("model_returend", BinaryType(), True),
                       StructField("Accuracy",FloatType(), True),
                       StructField("AUC",FloatType(), True)])
  
    
    # make training function
    #======================
    # this function will not take class attributes , so we will need to give it local/enclosing variables
    if self.stratified:
      group_column = self.group_column
    else:
      group_column= 'na'
    
    target = self.target_variable
    sessionid = self.session_id
    gpu = self.gpu_training
    model_name = self.model_name
    
    # if custome model only then make cp
    if model_name != "Auto Select":
      cp_create_model = self.custom_parameters
    
    # if user wants stratify the we do the apply in pandas way
    if self.stratified:
      start = time.perf_counter()
      def internal_training(key,pdf):
        # get the sample id
        id = pdf[group_column].iloc[0]
        # create a smaller dataset
        #small = pdf.head(10000)
        #call setup
        #cls1= setup(small.drop([group_column],axis=1), target = target, session_id=sessionid,silent=True,html=False,preprocess=False,use_gpu=gpu)
        cls1= setup(pdf.drop([group_column],axis=1), target = target, session_id=sessionid,silent=True,html=False,preprocess=False,use_gpu=gpu)
        # Now change the it with original (since we do not need preprocessing)
        #set_config('X_train',pdf.drop([group_column,target],axis=1))
        #set_config('y_train',pdf[target])

        #Modeling
        # if create model is selected
        if model_name == "Auto Select":
          model = compare_models(n_select=1,cross_validation=False) 
        else:
          model = create_model(model_name ,cross_validation=False,**cp_create_model) # while unpacking the custom parameters



        # try to get AUC
        try:
          prediction_prob = model.predict_proba (pdf.drop([group_column,target],axis=1))
          AUC = m.roc_auc_score(pdf[target],prediction_prob[:, 1])
        except:
          AUC = 0.0


        # get prediction & accuracy
        prediction = model.predict(pdf.drop([group_column,target],axis=1))  
        accuracy = m.accuracy_score(y_true=pdf[target],y_pred=prediction)
        # convert to df
        return_df = pd.DataFrame(dict(model_id = id,model_returend = [pickle.dumps(model)], Accuracy = accuracy, AUC = AUC))


        return(return_df)


      # Call the training function with sampled data
      #=============================================
      return_internal_training  = self.df_sample.groupby(self.group_column).applyInPandas(internal_training, schema=self.internal_training_schema)
      return_internal_training.cache()
      if self._count:
        return_internal_training.count()
      end = time.perf_counter()
      print("internal training took {0} minutes to run ...".format((end-start)/60))
      return_internal_training.cache()
      self.internal_training_results = return_internal_training
    
    # if user do not want to stratify  
    else:
      
      start = time.perf_counter()
      def internal_training(iterator: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:  
        for pdf in iterator:
          # create a smaller dataset
          #small = pdf.head(10000)
          #call setup
          #cls1= setup(small, target = target, session_id=sessionid,silent=True,html=False,preprocess=False,use_gpu=gpu)
          cls1= setup(pdf, target = target, session_id=sessionid,silent=True,html=False,preprocess=False,use_gpu=gpu)
          # Now change the it with original (since we do not need preprocessing)
          #set_config('X_train',pdf.drop([target],axis=1))
          #set_config('y_train',pdf[target])

          #Modeling
          # if create model is selected
          if model_name == "Auto Select":
            model = compare_models(n_select=1,cross_validation=False) 
          else:
            model = create_model(model_name ,cross_validation=False,**cp_create_model) # while unpacking the custom parameters

          # try to get AUC
          try:
            prediction_prob = model.predict_proba (pdf.drop([target],axis=1))
            AUC = m.roc_auc_score(pdf[target],prediction_prob[:, 1])
          except:
            AUC = 0.0

          # get prediction & accuracy
          prediction = model.predict(pdf.drop([target],axis=1))  
          accuracy = m.accuracy_score(y_true=pdf[target],y_pred=prediction)
          
          # convert to df, give same name as itreator element
          # for now we will keep model id as 0 , alter we will fill it by inremental index
          pdf = pd.DataFrame(dict(model_id = 0,model_returend = [pickle.dumps(model)], Accuracy = accuracy, AUC = AUC))
          # retuen this portion 
          yield pdf
      
      # Call the training function with sampled data
      #=============================================
      return_internal_training  = self.df_sample.mapInPandas(internal_training, schema=self.internal_training_schema)
      # extra step to fill the model_id column with increasing ID numbers
      return_internal_training = return_internal_training.withColumn("model_id",monotonically_increasing_id())
      window = Window.orderBy(col('model_id'))
      return_internal_training=return_internal_training.withColumn('model_id', row_number().over(window))
      return_internal_training = return_internal_training.withColumn('model_id',expr("model_id - 1"))
      return_internal_training.cache()
      
      # id is not unique index 
      if self._count:
        return_internal_training.count()
      end = time.perf_counter()
      print("internal training took {0} minutes to run ...".format((end-start)/60))

      self.internal_training_results = return_internal_training
      
      

    # Collect trained models in a dic
    #================================
    start = time.perf_counter()
    dict_models ={}
    for r in return_internal_training.rdd.collect():
      dict_models[r['model_id']] = pickle.loads(r['model_returend'])
    end = time.perf_counter()
    print("collecting models took {0} minutes to run ...".format((end-start)/60))
    spark.sparkContext.broadcast(dict_models)
    self.internal_trained_models = dict_models
    
    # Apply the second Level training 
    #================================
    #================================
    
    
    # First, creat a schema for training
    # outcome (predictions)
    #==================================
    
    # this has to be flexable, depending upon the 
    # batches we made 
    # easiest way is to creat a dummy pandas df 
    # with required column name and data types (always float)
    dummy = pd.DataFrame(dict(zip (list(range(self.batch_size)) , [[i*1.0,i*1.0] for i in range(self.batch_size)])))
    # add one extra column named labels
    dummy['label'] = 0.0
    # get the schema
    schema2 = spark.createDataFrame(dummy).schema
    self._useful_schema =  spark.createDataFrame(dummy).schema
    
    # creat another pandas udf 
    # for generating predictions
    # on entire training  data set
    #===========================
    # this will not take any instances
    def training_prediction_all(iterator: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
      for pdf in iterator:
        res = pd.DataFrame()
        for k in dict_models.keys():
          try:
            pre = dict_models[k].predict_proba(pdf.drop([group_column,target],axis=1,errors='ignore'))
            res[str(k)] = pre[:, 1]
          except:
            pre = dict_models[k].predict(pdf.drop([group_column,target],axis=1,errors='ignore'))
            res[str(k)] = pre

        # put the prediction with the results
        res['label'] = pdf[target]
        yield (res)
    
    start = time.perf_counter()
    return_internal_training_second = self.df_sample.mapInPandas(training_prediction_all, schema=schema2)
    if self._count:
      return_internal_training_second.count()
    end = time.perf_counter()
    
    print("predicion on entire training data took {0} minutes to run ...".format((end-start)/60))
    
    #Validation Predictionson
    # ========================
    #return predictions on entire validation data 
    # Do we need this ? remmeber we have OOB predictions from training data
    def validation_prediction_all(iterator: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
      for pdf in iterator:
        res = pd.DataFrame()
        for k in dict_models.keys():
          try:
            pre = dict_models[k].predict_proba(pdf.drop([group_column,target],axis=1,errors='ignore'))
            res[str(k)] = pre[:, 1]
          except:
            pre = dict_models[k].predict(pdf.drop([group_column,target],axis=1,errors="ignore"))
            res[str(k)] = pre

        # put the prediction with the results
        res['label'] = pdf[target]
        yield (res)
    
    start = time.perf_counter()
    return_internal_validation_second = self.validation_data.mapInPandas(validation_prediction_all, schema=schema2)
    if self._count:
      return_internal_validation_second.count()
    end = time.perf_counter()
    print("predicion on entire validation data took {0} minutes to run ...".format((end-start)/60))
    
    #Now train on second dataset with spark
    # # only IF Blend is set to True
    # So far only selecting lr
    #=======================================
    if blend:
      mlib_lr = LogisticRegression()
      # make feature vectors
      self.vector_assembler = VectorAssembler(inputCols=[str(i) for i in range(self.batch_size)],outputCol='features')
      # now transform
      df_training = self.vector_assembler.transform(return_internal_training_second)
      df_validation = self.vector_assembler.transform(return_internal_validation_second)

      # only select feature vector and target column

      df_training = df_training.select("features","label")


      df_validation = df_validation.select("features","label")

      # now train
      start = time.perf_counter()
      self.spark_fitted_model = mlib_lr.fit(df_training)
      end = time.perf_counter()
      print("spark model fitting took {0} minutes to run ...".format((end-start)/60))

      start = time.perf_counter()    
      Evaluated = self.spark_fitted_model.evaluate(df_validation)
      end = time.perf_counter()
      print("spark model evaluation took {0} minutes to run ...".format((end-start)/60))

      # make a data frame
      start = time.perf_counter()
      results = pd.DataFrame(dict(Accuracy=[self.spark_fitted_model.summary.accuracy,Evaluated.accuracy],AUC=[self.spark_fitted_model.summary.areaUnderROC,Evaluated.areaUnderROC]),index=['Training','Validation'])
      end = time.perf_counter()
      print("spark results took {0} minutes to run ...".format((end-start)/60))

      #self.train_res = fitted_model
      self.val_res = Evaluated
      
      
      
    # # only IF Blend is set to False
    # Then simple average with soft voting
    # So far only selecting lr
    #=======================================
    if not self.blend_option:
      
      # FOR TRAINING DATA SET
      # we need the table plus two more columns (score & voted prediction)
      return_internal_training_second = return_internal_training_second.withColumn("Average_Score",lit(0.0))
      return_internal_validation_second = return_internal_validation_second.withColumn("Average_Score",lit(0.0))
      
      l = self.batch_size
      cols = return_internal_training_second.columns[:-3]
      self._useful_cols = return_internal_training_second.columns[:-3]
      def pandas_filter(iterator: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
          for pdf in iterator:
            pdf['Average_Score'] = pdf[cols].mean(axis=1)
            yield pdf
      
      # For Training
      #-------------
      start = time.perf_counter()
      training_predictions  = return_internal_training_second.mapInPandas(pandas_filter, schema=return_internal_training_second.schema)
      # get prediction logic
      training_predictions = training_predictions.withColumn("prediction",when(training_predictions.Average_Score>=self.prediction_threshold,1.0).otherwise(0.0))
      self.training_predictions = training_predictions
      if self._count:
        self.training_predictions.count()
      end = time.perf_counter()
      print("voting on training data took {0} minutes to run ...".format((end-start)/60))
      
      
      # For Validation
      #---------------
      start = time.perf_counter()
      validation_predictions  = return_internal_validation_second.mapInPandas(pandas_filter, schema=return_internal_validation_second.schema)
      # get prediction logic
      validation_predictions = validation_predictions.withColumn("prediction",when(validation_predictions.Average_Score>=self.prediction_threshold,1.0).otherwise(0.0))
      self.validation_predictions = validation_predictions
      if self._count:
        self.validation_predictions.count()
      end = time.perf_counter()
      print("voting on validation data took {0} minutes to run ...".format((end-start)/60))
      
      # Accuracy, recall, precision , F1 will be calculated through multicalss evaluator 
      # AUC will be calculated through sklearn 
      training_accuracy = mult(predictionCol='prediction',probabilityCol='Average_Score',metricName='accuracy').evaluate(self.training_predictions)
      validation_accuracy =  mult(predictionCol='prediction',probabilityCol='Average_Score',metricName='accuracy').evaluate(self.validation_predictions)
      # define AUC function
      
      #@pandas_udf("double")
      #def pandas_auc(label: pd.Series,probability: pd.Series) -> float:
      #  score = roc_auc_score(y_true= label, y_score=probability)
      #  return score
      
      #training_auc = self.training_predictions.select(pandas_auc('label','Average_Score')).first()[0]
      #validation_auc = self.validation_predictions.select(pandas_auc('label','Average_Score')).first()[0]
      
      sch = spark.createDataFrame(pd.DataFrame(dict(auc=[0.0]))).schema
      def pandas_auc(iterator: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
        for pdf in iterator:
          auc = m.roc_auc_score(pdf.label,pdf.Average_Score)
          report = pd.DataFrame(dict(auc=[auc]))
          yield report

      training_auc_table = self.training_predictions.mapInPandas(pandas_auc, schema=sch)
      validation_auc_table = self.validation_predictions.mapInPandas(pandas_auc, schema=sch)
      
      # Now take the median
      training_auc = training_auc_table.approxQuantile('auc',[0.50],0)[0]
      validation_auc = validation_auc_table.approxQuantile('auc',[0.50],0)[0]
      
      
      #m_precision = m(predictionCol='Prediction',probabilityCol='Average_score',metricName='precisionByLabel').evaluate((cls.training_predictions))
      #recall=  m(predictionCol='Prediction',probabilityCol='Average_score',metricName='weightedRecall').evaluate((cls.training_predictions))
      start = time.perf_counter()
      results = pd.DataFrame(dict(Accuracy=[training_accuracy,validation_accuracy],AUC=[training_auc,validation_auc]),index=['Training','Validation'])
      end = time.perf_counter()
      print("results took {0} minutes to run ...".format((end-start)/60))
      
    
        
    return(results)
    
    
  #predict functio
  def predict(self,data:DataFrame):
    
    # Predictionson with Internal Trained Models
    # ========================
    #return predictions on entire validation data 
    # Do we need this ? remmeber we have OOB predictions from training data
    dict_models = self.internal_trained_models
    def test_prediction_all(iterator: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
      for pdf in iterator:
        res = pd.DataFrame()
        for k in dict_models.keys():
          try:
            pre = dict_models[k].predict_proba(pdf)
            res[str(k)] = pre[:, 1]
          except:
            pre = dict_models[k].predict(pdf)
            res[str(k)] = pre

        # put the prediction with the results
        #res['label'] = pdf[target]
        yield (res)
    
    start = time.perf_counter()
    return_internal_test_second = data.mapInPandas(test_prediction_all, schema=self._useful_schema[:-1])# dont need the lable column, remove last
    if self._count:
      return_internal_test_second.count()
    end = time.perf_counter()
    print("predicion on entire validation data took {0} minutes to run ...".format((end-start)/60))
    
    
    #Final Predictions
    #=================
    # Without Blending
    #-----------------
    
    # # only IF Blend is set to False
    # Then simple average with soft voting
    # So far only selecting lr
    #=======================================
    if not self.blend_option:
      
      # we need the table plus two more columns (score & voted prediction)
      return_internal_test_second = return_internal_test_second.withColumn("Average_Score",lit(0.0))
      
      
      l = self.batch_size
      cols = self._useful_cols
      def pandas_predict(iterator: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
          for pdf in iterator:
            pdf['Average_Score'] = pdf[cols].mean(axis=1)
            yield pdf
      

      # For Validation
      #---------------
      start = time.perf_counter()
      test_predictions  = return_internal_test_second.mapInPandas(pandas_predict, schema=return_internal_test_second.schema)
      # get prediction logic
      test_predictions = test_predictions.withColumn("Prediction",when(test_predictions.Average_Score>=self.prediction_threshold,1.0).otherwise(0.0))
      self.test_predictions = test_predictions
      if self._count:
        self.test_predictions.count()
      end = time.perf_counter()
      print("voting on validation data took {0} minutes to run ...".format((end-start)/60))
      
      # put the predictions with the test data
      
      # add id Column to data  and predictions
      transformed = data.withColumn("sudo_id",monotonically_increasing_id())
      test_predictions = test_predictions.withColumn("sudo_id",monotonically_increasing_id())
      transformed = transformed.join(test_predictions.select("sudo_id","Average_Score","Prediction"),"sudo_id",'inner')
      transformed = transformed.sort("sudo_id")
      transformed = transformed.drop('sudo_id')
       
      
   # Without Blending
    #-----------------
    if self.blend_option :
      
      
      # get features through vector_assembler
      df_test = self.vector_assembler.transform(return_internal_test_second)

      # only select feature vector and target column
      #df_test = df_test.select("features")

      start = time.perf_counter()    
      transformed = self.spark_fitted_model.transform(df_test)
      # drop features and raw_prediction columns
      transformed = transformed.drop('features','rawPrediction')
      end = time.perf_counter()
      print("spark model evaluation took {0} minutes to run ...".format((end-start)/60))
      
        
    return(transformed)
    
  

In [0]:
# 500 M records, 5 columns , partition = 100 , batch size of 8 , with stratified sampling - 15 minuts 
cls = Binary_Classifier(df_train_spark,'stabf',batch_size=8,gpu_training=True,validation_data=df_val_spark,stratified=True,_count=False)
cls.select_model('catboost',random_state=32)
# cls.auto_select_model()
cls.fit_predict()

sampling took 0.006563480816669198 minutes to run ...
internal training took 0.025333996599999863 minutes to run ...
collecting models took 21.41453289061667 minutes to run ...
predicion on entire training data took 0.0012416766999952718 minutes to run ...
predicion on entire validation data took 0.0012405817499939076 minutes to run ...
voting on training data took 0.000755532566669596 minutes to run ...
voting on validation data took 0.0008545916499997474 minutes to run ...
results took 1.3621066667231693e-05 minutes to run ...
Out[8]:

,Accuracy,AUC
Training,0.963911,0.988311
Validation,0.963896,0.988396


In [0]:
# 500 M records, 5 columns , partition = 100 , batch size of 8 , with stratified sampling - 15 minuts 
cls = Binary_Classifier(df_train_spark,'stabf',batch_size=8,gpu_training=True,validation_data=df_val_spark,stratified=True,_count=False)
cls.select_model('catboost',random_state=32)
# cls.auto_select_model()
cls.fit_predict(True)

sampling took 0.0045198520999899 minutes to run ...
internal training took 0.004152083366670922 minutes to run ...
collecting models took 21.24591591425001 minutes to run ...
predicion on entire training data took 0.0011733084333324466 minutes to run ...
predicion on entire validation data took 0.0010869986499983497 minutes to run ...
spark model fitting took 40.75947358878332 minutes to run ...
spark model evaluation took 0.002946915433343141 minutes to run ...
spark results took 26.64752629525001 minutes to run ...
Out[6]:

,Accuracy,AUC
Training,0.937888,0.980005
Validation,0.937771,0.979983


In [0]:
pred = cls.predict(df_test_spark)
pred.show()

predicion on entire validation data took 0.0010537725833273726 minutes to run ...
spark model evaluation took 0.0005093644333404276 minutes to run ...
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
 0| 1| 2| 3| 4| 5| 6| 7| probability|prediction|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
 0.8039982412645102| 0.8095844698137049| 0.8637804854920329| 0.8035090320305771| 0.8584732907704945| 0.8546912566621564| 0.8030030150740193| 0.8628791454213721|[0.09732997548039...| 1.0|
 0.5286353347723599| 0.5360103426622594| 0.5289405123830574| 0.5374267043146052| 0.5369458034554954| 0.5391401848339715| 0.54120246706537| 0.5428652558799054|[0.49924994403778...| 1.0|
0.001827938746603...|0.001523183780093487|0.001837385654185...|0.001891060219742428|0.001419697542539...|0.001585262318481...|0.001524408802998...|0.001619554172164434|[0.98392375553122...| 0.0|
 0.886636185709677| 0.8854852350641469| 0.8725645157232346| 0.8822159357666168| 0.8758465401575011| 0.8805158596010341| 0.8743793913655752| 0.8826502770367384|[0.06593817980525...| 1.0|
0.005131609867173326| 0.00605719784111064|0.005973730013257...|0.005720362632196337|0.006219950843202496|0.005859177604340725|0.005758402715259161|0.005721510515620941|[0.98344200497484...| 0.0|
0.002825212879426...|0.002605177020185...|0.002383021847894...|0.002940432713977...|0.003233942413313...|0.002351531619944...|0.002961795925345...|0.002608507910463...|[0.98379433213678...| 0.0|
 0.9610987788681647| 0.9620316501902205| 0.9690280319344631| 0.9657579716727429| 0.9622345199665651| 0.9571184069841104| 0.9630046908804972| 0.9624093140963307|[0.03666380077116...| 1.0|
0.038635608105073986| 0.03703550497709509|0.039724440241230806| 0.04159070248718897| 0.04387035085338393| 0.03925707314372965| 0.03564169817636674| 0.0373299416265959|[0.97879329068820...| 0.0|
 0.17482459194976036| 0.17991136679142794| 0.16460795226693595| 0.16721884132149503| 0.16552709515862446| 0.1777584927453197| 0.17643687369655164| 0.15350383737257167|[0.94423440049005...| 0.0|
0.002435140421165...|0.002254395013671...|0.002480009057196443|0.002848609050124787|0.002558951778661...|0.002517708768828...|0.002667506308190...|0.002786454100611...|[0.98380826915299...| 0.0|
0.025103318808864464| 0.0246381042499408|0.023862878323393993| 0.02971071499587019| 0.02401080636556647|0.027901550883650796|0.025328134443956263| 0.0351591369891937|[0.98029331189710...| 0.0|
 0.8626403662509474| 0.8624905879884524| 0.8652958771415337| 0.8682890268399338| 0.8643491124749216| 0.86968682948581| 0.8621660843342362| 0.8578596556256661|[0.07522609480089...| 1.0|
 0.9819354720953529| 0.9821892509591078| 0.9828704269467726| 0.9823206020656455| 0.9836510059322819| 0.9824787190765378| 0.9832001156029774| 0.9826973751804398|[0.03148521040611...| 1.0|
 0.9724468297829784| 0.9724788621236851| 0.9714630965962548| 0.9760934745556337| 0.969571757995214| 0.9740314750152943| 0.9704441766338419| 0.969569775384422|[0.03402841509513...| 1.0|
 0.7403149740964311| 0.7439527006497871| 0.7479719172293178| 0.742675243239651| 0.7524986032175653| 0.7315422075651514| 0.7655496899745028| 0.7640840816868667|[0.16247312107386...| 1.0|
 0.4206577593182942| 0.4151566074892981| 0.3985426723778618| 0.3986818115749684| 0.3661316325799739| 0.4046152409014322| 0.393467324584535| 0.40263648023665805|[0.73404120964262...| 0.0|
 0.9034657451552687| 0.9031429172431774| 0.8928299210881702| 0.8932710203646274| 0.893754487853807| 0.9047251798658984| 0.8964353935723045| 0.8929392179510274|[0.05861166021328...| 1.0|
0.007220983666730803|0.006566637722871454|0.006841793147235695|0.007522448308593958|0.007217918411276...|0.006124636357309...|0.005931330043112979|0.006371947554209808|[0.983

In [0]:
cls.internal_training_results.show()

+--------+--------------------+----------+----------+
model_id| model_returend| Accuracy| AUC|
+--------+--------------------+----------+----------+
 1|[80 03 63 63 61 7...| 0.9377935|0.97993726|
 6|[80 03 63 63 61 7...| 0.9378944|0.97992396|
 3|[80 03 63 63 61 7...|0.93785733| 0.9799291|
 5|[80 03 63 63 61 7...| 0.9377904| 0.9799284|
 4|[80 03 63 63 61 7...| 0.9378538| 0.9799203|
 7|[80 03 63 63 61 7...|0.93781394|0.97991914|
 2|[80 03 63 63 61 7...| 0.937788|0.97989845|
 0|[80 03 63 63 61 7...|0.93785924|0.97991467|
+--------+--------------------+----------+----------+

In [0]:
# 100 M records, 5 columns , partition = 100 , batch size of 8 , with stratified sampling - 15 minuts 
cls = Binary_Classifier(df_train_spark,'stabf',batch_size=1,gpu_training=True,validation_data=df_val_spark,stratified=True,_count=False)
cls.select_model('catboost',random_state=32)
# cls.auto_select_model()
cls.fit_predict()

sampling took 0.002555131633319737 minutes to run ...
internal training took 0.00858999426667045 minutes to run ...
collecting models took 22.674851706716677 minutes to run ...
predicion on entire training data took 0.0005736577166620312 minutes to run ...
predicion on entire validation data took 0.0005165601166481793 minutes to run ...
voting on training data took 0.0007027489999927638 minutes to run ...
voting on validation data took 0.0006814965666611291 minutes to run ...

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-75529488055979> in <module> 
 3 cls . select_model ( 'catboost' , random_state = 32 ) 
 4 # cls.auto_select_model() 
 ----> 5 cls . fit_predict ( ) 

 <command-1899558221412544> in fit_predict (self, blend) 
 418 # Accuracy, recall, precision , F1 will be calculated through multicalss evaluator 
 419 # AUC will be calculated through sklearn 
 --> 420 training_accuracy = mult ( predictionCol = 'prediction' , probabilityCol = 'Average_Score' , metricName = 'accuracy' ) . evaluate ( self . training_predictions ) 
 421 validation_accuracy = mult ( predictionCol = 'prediction' , probabilityCol = 'Average_Score' , metricName = 'accuracy' ) . evaluate ( self . validation_predictions ) 
 422 # define AUC function 

 /databricks/spark/python/pyspark/ml/evaluation.py in evaluate (self, dataset, params) 
 70 return self . copy ( params ) . _evaluate ( dataset ) 
 71 else : 
 ---> 72 return self . _evaluate ( dataset ) 
 73 else : 
 74 raise ValueError ( "Params must be a param map but got %s." % type ( params ) ) 

 /databricks/spark/python/pyspark/ml/evaluation.py in _evaluate (self, dataset) 
 100 """
 101 self . _transfer_params_to_java ( ) 
 --> 102 return self . _java_obj . evaluate ( dataset . _jdf ) 
 103 
 104 def isLargerBetter ( self ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 125 def deco ( * a , ** kw ) : 
 126 try : 
 --> 127 return f ( * a , ** kw ) 
 128 except py4j . protocol . Py4JJavaError as e : 
 129 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o3339.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 13.0 failed 4 times, most recent failure: Lost task 7.3 in stage 13.0 (TID 1063, 10.70.176.14, executor 5): java.lang.IllegalStateException: Value at index is null
	at org.apache.arrow.vector.Float8Vector.get(Float8Vector.java:115)
	at org.apache.spark.sql.vectorized.ArrowColumnVector$DoubleAccessor.getDouble(ArrowColumnVector.java:349)
	at org.apache.spark.sql.vectorized.ArrowColumnVector.getDouble(ArrowColumnVector.java:96)
	at org.apache.spark.sql.vectorized.ColumnarBatchRow.getDouble(ColumnarBatch.java:211)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.apply(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.apply(Unknown Source)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:459)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage8.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:732)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.apache.spark.util.collection.ExternalSorter.insertAll(ExternalSorter.scala:192)
	at org.apache.spark.shuffle.sort.SortShuffleWriter.write(SortShuffleWriter.scala:62)
	at org.apache.spark.shuffle.Shuf

In [0]:
# 100 M records, 5 columns , partition = 100 , batch size of 8 , with stratified sampling - 15 minuts 
cls = Binary_Classifier(df_train_spark,'stabf',batch_size=4,gpu_training=True,validation_data=df_val_spark,stratified=True,_count=False)
cls.select_model('catboost',random_state=32)
# cls.auto_select_model()
cls.fit_predict()

sampling took 0.0028014029833381454 minutes to run ...
internal training took 0.0019227729166838496 minutes to run ...
collecting models took 6.312385410700032 minutes to run ...
predicion on entire training data took 0.0007260016499155123 minutes to run ...
predicion on entire validation data took 0.0007115689500399943 minutes to run ...
voting on training data took 0.00044263568330886 minutes to run ...
voting on validation data took 0.00040764553341432475 minutes to run ...
results took 1.3489399983275992e-05 minutes to run ...
Out[9]:

,Accuracy,AUC
Training,0.950541,0.987517
Validation,0.950520,0.987670


In [0]:
# 100 M records, 5 columns , partition = 100 , batch size of 8 , with stratified sampling - 15 minuts 
cls = Binary_Classifier(df_train_spark,'stabf',batch_size=8,gpu_training=True,validation_data=df_val_spark,stratified=True,_count=False)
cls.select_model('catboost',random_state=32)
# cls.auto_select_model()
cls.fit_predict()

sampling took 0.0029803089833270254 minutes to run ...
internal training took 0.0033506093833542156 minutes to run ...
collecting models took 4.073708159599968 minutes to run ...
predicion on entire training data took 0.001267922783335962 minutes to run ...
predicion on entire validation data took 0.0011179126999801762 minutes to run ...
voting on training data took 0.0005880842333378192 minutes to run ...
voting on validation data took 0.0004971565499848414 minutes to run ...
results took 1.3271100033307449e-05 minutes to run ...
Out[3]:

,Accuracy,AUC
Training,0.950664,0.987557
Validation,0.950625,0.987699


In [0]:
# 100 M records, 5 columns , partition = 100 , batch size of 8 , with stratified sampling - 15 minuts 
cls = Binary_Classifier(df_train_spark,'stabf',batch_size=8,gpu_training=True,validation_data=df_val_spark,stratified=True,_count=False)
cls.select_model('catboost',random_state=32)
# cls.auto_select_model()
cls.fit_predict()

sampling took 0.003313067599992792 minutes to run ...
internal training took 0.003804996216664828 minutes to run ...
collecting models took 3.979585134949957 minutes to run ...
predicion on entire training data took 0.0011429104166381875 minutes to run ...
predicion on entire validation data took 0.001112975050030703 minutes to run ...
voting on training data took 0.000628397733332046 minutes to run ...
voting on validation data took 0.0005169075666344725 minutes to run ...
results took 1.3532733282772824e-05 minutes to run ...
Out[29]:

,Accuracy,AUC
Training,0.983273,0.994413
Validation,0.983133,0.994385


In [0]:
# spark.sql.execution.arrow.maxRecordsPerBatch =20000

spark.conf.set('spark.sql.execution.arrow.maxRecordsPerBatch', 20000)

In [0]:
cls = Binary_Classifier(df_train_spark,'stabf',batch_size=8,gpu_training=True,validation_data=df_val_spark,stratified=False)
cls.select_model('catboost',random_state=32)
# cls.auto_select_model()
cls.fit_predict()

internal training took 130.90320117775 minutes to run ...

In [0]:
cls = Binary_Classifier(df_train_spark,'stabf',batch_size=9,gpu_training=True,validation_data=df_val_spark)
cls.select_model('catboost',random_state=32)
# cls.auto_select_model()
cls.fit_predict()

processing took 0.9166423180333367 minutes to run ...
internal training took 24.79490470063333 minutes to run ...
collecting models took 24.851206623700016 minutes to run ...
predicion on entire training data took 2.4363147199833293 minutes to run ...
predicion on entire validation data took 0.09960218963333319 minutes to run ...
voting on training data took 2.517149295466667 minutes to run ...
voting on validation data took 0.1186590748500142 minutes to run ...
results took 1.3152766647787455e-05 minutes to run ...
Out[9]:

,Accuracy,AUC
Training,0.973097,0.987587
Validation,0.972917,0.987546


In [0]:
cls = Binary_Classifier(df_train_spark,'stabf',batch_size=3,gpu_training=True,validation_data=df_val_spark)
cls.select_model('catboost',random_state=32)
# cls.auto_select_model()
cls.fit_predict()

processing took 1.9130208439 minutes to run ...
internal training took 35.60024464343334 minutes to run ...
collecting models took 35.750343953716666 minutes to run ...
predicion on entire training data took 2.144660063049999 minutes to run ...
predicion on entire validation data took 0.10821835603333056 minutes to run ...
voting on training data took 2.235138334816672 minutes to run ...
voting on validation data took 0.11874302953333427 minutes to run ...
results took 1.3797750004111246e-05 minutes to run ...
Out[4]:

,Accuracy,AUC
Training,0.973009,0.987512
Validation,0.972846,0.987512


In [0]:
cls = Binary_Classifier(df_train_spark,'stabf',batch_size=3,gpu_training=True,validation_data=df_val_spark)
cls.select_model('catboost',random_state=32)
# cls.auto_select_model()
cls.fit_predict()

processing took 0.7812878509666613 minutes to run ...
internal training took 38.47075304411665 minutes to run ...
collecting models took 38.732428989966685 minutes to run ...
predicion on entire training data took 2.0825072821999737 minutes to run ...
predicion on entire validation data took 0.03759656634999071 minutes to run ...
spark model fitting took 5.650339553099972 minutes to run ...
spark model evaluation took 0.003132877216679238 minutes to run ...
spark results took 5.139419421650018 minutes to run ...
 Accuracy AUC
Training 0.902101 0.959963
Validation 0.901571 0.959417

In [0]:
cls = Binary_Classifier(df_train_spark,'stabf',batch_size=8,gpu_training=True,validation_data=df_val_spark)
cls.select_model('catboost',random_state=32)
# cls.auto_select_model()
cls.fit_predict()

processing took 3.1833058083333525 minutes to run ...
internal training took 23.64467214589998 minutes to run ...
collecting models took 17.504474079266704 minutes to run ...
predicion on entire training data took 4.52341690171664 minutes to run ...
predicion on entire validation data took 0.08390689539998372 minutes to run ...
spark model fitting took 9.757227202600006 minutes to run ...
spark model evaluation took 0.0026104315499348254 minutes to run ...
spark results took 11.236492528516708 minutes to run ...
 Accuracy AUC
Training 0.971559 0.990228
Validation 0.971661 0.990371

In [0]:
wcls.internal_training_res.show()

+--------+--------------------+----------+---------+
model_id| model_returend| Accuracy| AUC|
+--------+--------------------+----------+---------+
 1|[80 03 63 63 61 7...| 0.9471907|0.9823966|
 2|[80 03 63 63 61 7...|0.94718707| 0.982355|
 0|[80 03 63 63 61 7...|0.94721574|0.9823801|
+--------+--------------------+----------+---------+

In [0]:
|model_id|      model_returend|  Accuracy|      AUC|
+--------+--------------------+----------+---------+
|       0|[80 03 63 73 6B 6...| 0.8076923|0.8469733|
|       1|[80 03 63 73 6B 6...| 0.8012048|0.8474444|
|       2|[80 03 63 73 6B 6...|0.82175225|0.8442235|
+--------+--------------------+----------+---------+


Out[25]: {'random_state': 32}

In [0]:
def exp(a=1, **kwargs):
  print(a,kwargs)

In [0]:
exp(3, **cls.custom_parameters)

3 {'random_state': 32}

In [0]:
dict(zip (list(range(5)) , [i*1.0 for i in range(5)]))

Out[59]: {0: 0.0, 1: 1.0, 2: 2.0, 3: 3.0, 4: 4.0}

In [0]:
from pycaret.classification import *

In [0]:
s = setup(data,'stabf',0.99,silent=True,preprocess=False,use_gpu=True)

IntProgress(value=0, description='Processing: ', max=3)
 
 
Initiated . . . . . . . . . . . . . . . . . . 19:10:50
Status . . . . . . . . . . . . . . . . . . Loading Dependencies
 
 
Initiated . . . . . . . . . . . . . . . . . . 19:10:50
Status . . . . . . . . . . . . . . . . . . Preparing Data for Modeling
 
 
Initiated . . . . . . . . . . . . . . . . . . 19:10:50
Status . . . . . . . . . . . . . . . . . . Preparing Data for Modeling

--------------------------------------------------------------------------- 
 MemoryError Traceback (most recent call last)
 <command-75529488056011> in <module> 
 ----> 1 s = setup ( data , 'stabf' , 0.99 , silent = True , preprocess = False , use_gpu = True ) 

 /databricks/python/lib/python3.7/site-packages/pycaret/classification.py in setup (data, target, train_size, test_data, preprocess, imputation_type, iterative_imputation_iters, categorical_features, categorical_imputation, categorical_iterative_imputer, ordinal_features, high_cardinality_features, high_cardinality_method, numeric_features, numeric_imputation, numeric_iterative_imputer, date_features, ignore_features, normalize, normalize_method, transformation, transformation_method, handle_unknown_categorical, unknown_categorical_method, pca, pca_method, pca_components, ignore_low_variance, combine_rare_levels, rare_level_threshold, bin_numeric_features, remove_outliers, outliers_threshold, remove_multicollinearity, multicollinearity_threshold, remove_perfect_collinearity, create_clusters, cluster_iter, polynomial_features, polynomial_degree, trigonometry_features, polynomial_threshold, group_features, group_names, feature_selection, feature_selection_threshold, feature_selection_method, feature_interaction, feature_ratio, interaction_threshold, fix_imbalance, fix_imbalance_method, data_split_shuffle, data_split_stratify, fold_strategy, fold, fold_shuffle, fold_groups, n_jobs, use_gpu, custom_pipeline, html, session_id, log_experiment, experiment_name, log_plots, log_profile, log_data, silent, verbose, profile, profile_kwargs) 
 652 verbose = verbose , 
 653 profile = profile , 
 --> 654 profile_kwargs = profile_kwargs , 
 655 )
 656 

 /databricks/python/lib/python3.7/site-packages/pycaret/internal/tabular.py in setup (data, target, ml_usecase, available_plots, train_size, test_data, preprocess, imputation_type, iterative_imputation_iters, categorical_features, categorical_imputation, categorical_iterative_imputer, ordinal_features, high_cardinality_features, high_cardinality_method, numeric_features, numeric_imputation, numeric_iterative_imputer, date_features, ignore_features, normalize, normalize_method, transformation, transformation_method, handle_unknown_categorical, unknown_categorical_method, pca, pca_method, pca_components, ignore_low_variance, combine_rare_levels, rare_level_threshold, bin_numeric_features, remove_outliers, outliers_threshold, remove_multicollinearity, multicollinearity_threshold, remove_perfect_collinearity, create_clusters, cluster_iter, polynomial_features, polynomial_degree, trigonometry_features, polynomial_threshold, group_features, group_names, feature_selection, feature_selection_threshold, feature_selection_method, feature_interaction, feature_ratio, interaction_threshold, fix_imbalance, fix_imbalance_method, transform_target, transform_target_method, data_split_shuffle, data_split_stratify, fold_strategy, fold, fold_shuffle, fold_groups, n_jobs, use_gpu, custom_pipeline, html, session_id, log_experiment, experiment_name, log_plots, log_profile, log_data, silent, verbose, profile, profile_kwargs, display) 
 1015 
 1016 # creating variables to be used later in the function 
 -> 1017 train_data = data_before_preprocess . copy ( ) 
 1018 if _is_unsupervised ( _ml_usecase ) : 
 1019 target = "UNSUPERVISED_DUMMY_TARGET" 

 /databricks/python/lib/python3.7/site-packages/pandas/core/generic.py in copy (self, deep) 
 5809 dtype : object
 5810 """
 -> 5811 data = self . _data . copy ( deep = deep ) 
 5812 return self . _constructor ( data ) . __finalize__ ( self ) 
 5813 

 /databricks/python/lib/python3.7/site-packages/pandas/core/internals/managers.py in copy (self, deep) 
 792 new_axes = list ( self . axes ) 
 793 
 --> 794 res = self . apply ( "copy" , deep = deep ) 
 795 res . axes = new_axes
 796 return res

 /databricks/python/lib/python3.7/site-packages/pandas/core/internals/managers.py in apply (self, f, filter, **kwargs) 
 440 a

In [0]:
m = create_model("catboost",cross_validation=False)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-75529488056012> in <module> 
 ----> 1 m = create_model ( "catboost" , cross_validation = False ) 

 /databricks/python/lib/python3.7/site-packages/pycaret/classification.py in create_model (estimator, fold, round, cross_validation, fit_kwargs, groups, verbose, **kwargs) 
 895 groups = groups , 
 896 verbose = verbose , 
 --> 897 ** kwargs , 
 898 )
 899 

 /databricks/python/lib/python3.7/site-packages/pycaret/internal/tabular.py in create_model_supervised (estimator, fold, round, cross_validation, predict, fit_kwargs, groups, refit, verbose, system, X_train_data, y_train_data, metrics, display, **kwargs) 
 2866 runtime_start = time . time ( ) 
 2867 
 -> 2868 available_estimators = set ( _all_models_internal . keys ( ) ) 
 2869 
 2870 if not fit_kwargs : 

 NameError : name '_all_models_internal' is not defined

In [0]:
m

Out[9]: <catboost.core.CatBoostClassifier at 0x7fc172029a50>

In [0]:
data.shape

Out[3]: (100000000, 6)

In [0]:
import pyarrow

In [0]:
pyarrow.__version__

Out[2]: '0.15.1'